In [1]:
from lxml import etree
import pandas as pd
from collections import Counter
import os
import glob
import re
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from numpy import array
import numpy as np

import subprocess


# Open Data

In [2]:

wdir = "./../../../sexual-annotation/"
file = "TEIBible" # "*.xml"
input_dir = "./../../../annotated/books/"
output_dir = "./../../../annotated/books_POS_MariaX/"

In [3]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [4]:
bible_root = etree.parse(wdir + file + ".xml").getroot()

In [5]:
bible_root

<Element {http://www.tei-c.org/ns/1.0}teiCorpus at 0x2a4851832c8>

In [6]:
namespaces = {'tei': 'http://www.tei-c.org/ns/1.0'}

In [7]:
mariax = pd.read_parquet("./../../../../../Doktorarbeit/dictionaries-for-annotation/mariaX/resulting-data/catalogues_only-tokens_df.parquet")

In [9]:
mariax

,a,aaronita,ab,aba,abab,ababillarse,abacería,abacorar,abacá,abad,...,últimas,último,úlula,únicamente,único,úrico,úrsido,útero,útil,ýpsilon
sublema,,,,,,,,,,,,,,,,,,,,,
abandonar,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abatir,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abeja,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abertura,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abiertamente,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yeso,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
yugo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zaherir,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
len(mariax.index.tolist())

1788

In [8]:
# Annotate Books

In [67]:
for book_path in glob.glob(input_dir + "/*"):
    #print(book_path)
    book_name = book_path[-7:-4]
    print(book_name)

    book_root = etree.parse(input_dir + book_name + ".xml").getroot()


    for verse in book_root.xpath(".//tei:seg", namespaces = namespaces):
        #verse_id = verse.xpath("./@xml:id")[0][0:-6]

        #content_verse = '\n<seg xml:id="' + verse_id + '.MariaX" corresp="#' + verse_id + '">\n'
        for sentence in verse.xpath("./tei:s", namespaces = namespaces):
            sentence_n = sentence.xpath("./@n", namespaces = namespaces)[0]

            #content_verse += '\n<s n="'  + sentence_n+ '">\n'
            for word in verse.xpath(".//tei:w", namespaces = namespaces):

                
                neclass = word.xpath("./@neclass")
                lemma = word.xpath("./@lemma")[0]
                n = word.xpath("./@n")[0]
                #if neclass != ["person"]:
                if lemma in mariax.columns.tolist():
                        #print(lemma)
                #       content_verse +=  '<w n="' + n + '" lemma="'  + lemma + '" mariax="' + '|'.join(mariax.loc[:,lemma][(mariax.loc[:,lemma] > 0)].index.tolist()) + '"/>\n' 
                    word.attrib["mariax"] =  '|'.join(mariax.loc[:,lemma][(mariax.loc[:,lemma] > 0)].index.tolist())
            #content_verse += "\n</s>"


        #content_book += content_verse+ '\n</seg>\n'

    #content_book += '\n</standOff>'

    book_root.getroottree().write(output_dir + book_name + '.xml', encoding="utf8")


1CH
1CO
1JO
1KI
1PE
1SA
1TH
1TI
2CH
2CO
2JO
2KI
2PE
2SA
2TH
2TI
3JO
ACT
AMO
COL
DAN
DEU
ECC
EPH
EST
EXO
EZE
EZR
GAL
GEN
HAB
HAG
HEB
HOS
ISA
JAM
JDG
JER
JOB
JOE
JOH
JON
JOS
JUD
LAM
LEV
LUK
MAL
MAR
MAT
MIC
NAH
NEH
NUM
OBA
PHI
PHM
PRO
PSA
REV
ROM
RUT
SON
TIT
ZEC
ZEP


# Extract Data

In [15]:
verses_lt = bible_root.xpath(".//tei:ab[@type='verse']/@xml:id", namespaces = namespaces)

In [23]:
verses_mariax = pd.DataFrame(index = verses_lt, columns = mariax.columns.tolist()).fillna(0)

In [25]:
verses_mariax.fillna(0, inplace=True)

In [26]:
verses_mariax

,a,aaronita,ab,aba,abab,ababillarse,abacería,abacorar,abacá,abad,...,últimas,último,úlula,únicamente,único,úrico,úrsido,útero,útil,ýpsilon
b.GEN.001.001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b.GEN.001.002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b.GEN.001.003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b.GEN.001.004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b.GEN.001.005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b.REV.022.017,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b.REV.022.018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b.REV.022.019,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b.REV.022.020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
books_lt = bible_root.xpath(".//tei:title[@type='idno']/tei:idno[@type='string']//text()", namespaces = namespaces)


In [18]:
books_lt

['GEN',
 'EXO',
 'LEV',
 'NUM',
 'DEU',
 'JOS',
 'JDG',
 'RUT',
 '1SA',
 '2SA',
 '1KI',
 '2KI',
 '1CH',
 '2CH',
 'EZR',
 'NEH',
 'EST',
 'JOB',
 'PSA',
 'PRO',
 'ECC',
 'SON',
 'ISA',
 'LAM',
 'JER',
 'EZE',
 'DAN',
 'HOS',
 'JOE',
 'AMO',
 'OBA',
 'JON',
 'MIC',
 'NAH',
 'HAB',
 'ZEP',
 'HAG',
 'ZEC',
 'MAL',
 'MAT',
 'MAR',
 'LUK',
 'JOH',
 'ACT',
 'ROM',
 '1CO',
 '2CO',
 'GAL',
 'EPH',
 'PHI',
 'COL',
 '1TH',
 '2TH',
 '1TI',
 '2TI',
 'TIT',
 'PHM',
 'HEB',
 'JAM',
 '1PE',
 '2PE',
 '1JO',
 '2JO',
 '3JO',
 'JUD',
 'REV']

In [85]:

output_dir = './../../../annotated/books_parquets_verses_Mariax/'

# = "./../../../annotated/books/"
input_dir = "./../../../annotated/books_POS_MariaX/"

In [114]:

verses_lt = bible_root.xpath(".//tei:ab[@type='verse']/@xml:id", namespaces = namespaces)
print(len(verses_lt))

verses_mariax_bible = pd.DataFrame(index = verses_lt, columns = mariax.index.unique().tolist()).fillna(0)

print(verses_mariax_bible.shape)


for book_element in bible_root.xpath(".//tei:TEI", namespaces = namespaces):

    book_name = book_element.xpath(".//tei:title[@type='idno']/tei:idno[@type='string']//text()", namespaces = namespaces)[0]
    print(book_name)


#for book_name in books_lt[0:3]:

    book_root = etree.parse(input_dir + book_name + ".xml").getroot()

    #print(book_root)
    for seg in book_root.xpath(".//tei:seg", namespaces = namespaces):
        verse_id = seg.xpath("./@xml:id")[0][0:-6]
        #print(verse_id)
        for w in seg.xpath(".//tei:w[@mariax]", namespaces = namespaces):
            #print(w.xpath("./@neclass"))
            if w.xpath("./@pos")[0] in ["verb", "noun", "adjective", "adverb"] and w.xpath("./@neclass") == []:
            #if w.xpath("./@pos") in ["verb", "noun", "adjective", "adverb"] and :
                #print(w.xpath("./@lemma"))
                #print(w.xpath("./@mariax"))
                for value_catalog in w.xpath("./@mariax")[0].split("|"):
                    #print(value_catalog)

                    try:

                        verses_mariax_bible.loc[verse_id, value_catalog] +=  1
                        #print(verses_mariax_book.loc[verse_id, value_catalog])
                    except:
                        print("error: ", value_catalog)

 
verses_mariax_bible.to_parquet(output_dir + "verses_mariax.parquet")





31065
(31065, 1783)
GEN
EXO
LEV
NUM
DEU
JOS
JDG
RUT
1SA
2SA
1KI
2KI
1CH
2CH
EZR
NEH
EST
JOB
PSA
PRO
ECC
SON
ISA
LAM
JER
EZE
DAN
HOS
JOE
AMO
OBA
JON
MIC
NAH
HAB
ZEP
HAG
ZEC
MAL
MAT
MAR
LUK
JOH
ACT
ROM
1CO
2CO
GAL
EPH
PHI
COL
1TH
2TH
1TI
2TI
TIT
PHM
HEB
JAM
1PE
2PE
1JO
2JO
3JO
JUD
REV


In [117]:
verses_mariax_bible.sum().sort_values(ascending=False).head(50)

tiempo          38031
hablar          33823
difícil         33237
existir         29869
cuenta          25978
representar     25581
baraja          24043
merecer         22865
intervenir      21483
importar        21155
superior        20762
experiencia     20743
equivocar       20490
armonía         20439
decir           20140
negocio         19485
sobresalir      18746
conversar       18705
alegre          17576
componer        17328
adecuado        17134
bueno           16578
suceder         16458
terminar        16335
igual           16296
negar           16206
vestir          16069
jugar           16009
callar          15803
imponer         15738
nombre          15736
esencia         15510
fútbol          15476
decidir         15225
riña            15201
efecto          15184
justo           14752
tener           14478
expresar        14466
solución        14244
tratar          14124
seco            14119
corresponder    14098
descubrir       13993
bien            13981
opinión   